In [1]:
import pickle
import os
import sys
import six
import json
import twitter

In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS Qualgo.json
from google.cloud import language
from google.cloud import language_v1beta2
from google.cloud.language_v1beta2 import enums
from google.cloud.language_v1beta2 import types

env: GOOGLE_APPLICATION_CREDENTIALS=C:\\Users\\Jimit\\Downloads\\Qualgo-d74f07b5ea27.json


In [3]:
if not os.path.exists('secret_twitter_credentials.pkl'):
    Twitter={}
    Twitter['Consumer Key'] = '---SECRET---'
    Twitter['Consumer Secret'] = '---SECRET---'
    Twitter['Access Token'] = '---SECRET---'
    Twitter['Access Token Secret'] = '---SECRET---'
    with open('secret_twitter_credentials.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

Now we store Twitter Application Credentials in a dictionary named "Twitter" with fields 'Access Token', 'Access Token Secret', 'Consumer Key' and 'Consumer Secret'. Note that this step is omitted over here for security reasons.

In [4]:
auth = twitter.oauth.OAuth(Twitter['Access Token'],
                           Twitter['Access Token Secret'],
                           Twitter['Consumer Key'],
                           Twitter['Consumer Secret'])

twitter_api = twitter.Twitter(auth=auth)

In [5]:
def contains_topic(text, topic): #Checks if the topic is in the text 
    text = text.lower()
    topic = topic.lower()
    if text.find(topic) != -1:
        return True
    else:
        return False

In [6]:
def entity_sentiment_text(text, topic): #Outputs relevant metrics of sentiment
    
    """Detects entity sentiment in the provided text."""
    client = language_v1beta2.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Pass in encoding type to get useful offsets in the response.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    for entity in result.entities:
        if entity.name == topic:
            dict = {}
            dict["magnitude"] = entity.sentiment.magnitude
            dict["score"] = entity.sentiment.score
            dict["salience"] = entity.salience
            return dict

In [7]:
example_text = 'Google, headquartered in Mountain View, unveiled the new Android phone at the Consumer Electronic Show.  Sundar Pichai said in his keynote that users love their new Android phones.'
#sentiment, entities = language_analysis(example_text)

print(entity_sentiment_text(example_text, "Google"))

{'magnitude': 0.0, 'score': 0.0, 'salience': 0.2569761574268341}


In [8]:
def topic_trend(topic): #Outputs a dictionary containing num of followers of poster and sentiment metrics
    search_results = twitter_api.search.tweets(q = topic, count = 100, lang = "en")
    statuses = search_results['statuses']
    table = []
    for status in statuses:
        if contains_topic(status['text'], topic):
            dict = {}
            value = entity_sentiment_text(status['text'], topic)
            num_followers = status['user']['followers_count']
            dict["value"] = value
            dict["num"] = num_followers
            table.append(dict)
    return table

In [9]:
def final_output(topic): #Outputs a sentiment associated with the search topic
    table = topic_trend(topic)
    lst_values = []
    aggregate = 0
    for i in table:
        if i["value"]:
            lst_values.append(i["value"]["score"] * i["num"])
            aggregate = aggregate + i["num"]
    final_value = sum(lst_values) / float(aggregate)
    if final_value >= 0.001:
        return "Positive"
    elif final_value < -0.001:
        return "Negative"
    else: 
        return "Neutral"

In [11]:
final_output("Terrorism")

'Negative'